_한국어로 기계번역됨_


# LangGraph 호출을 LangGraph Cloud 호출로 변환하는 방법


그래서 당신은 그래프와 로컬에서 상호작용하는 데 익숙하지만 이제 LangGraph 클라우드와 함께 배포했습니다. 코드베이스에서 LangGraph를 직접 호출하는 모든 부분을 LangGraph Cloud를 호출하도록 변경하려면 어떻게 해야 하나요? 이 노트북에는 LangGraph를 호출하는 것과 LangGraph Cloud를 호출하는 것을 쉽게 전환할 수 있도록 나란히 비교한 내용이 포함되어 있습니다.


## 설정


이 방법 안내서에서는 간단한 ReAct 에이전트를 사용할 것입니다. 또한 `agent.py` 및 `langgraph.json` 파일이 포함된 프로젝트를 설정해야 합니다. 설정 방법에 대한 내용은 [신속 시작](https://langchain-ai.github.io/langgraph/cloud/quick_start/#develop)을 참조하십시오.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


OPENAI_API_KEY:  ········


In [1]:
# this is all that's needed for the agent.py
from typing import Literal
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

model = ChatOpenAI(model_name="gpt-4o", temperature=0)
graph = create_react_agent(model, tools)


이제 langgraph 클라이언트를 설정하겠습니다. 클라이언트는 LangGraph Cloud 서버가 `localhost:8123`에서 실행되고 있다고 가정합니다.


In [10]:
from langgraph_sdk import get_client

client = get_client()


## 그래프 호출하기


아래의 예제는 LangGraph의 `CompiledGraph` 실행 가능 항목의 `.invoke() / .ainvoke()` 메서드를 미러링하는 방법을 보여줍니다. 즉, 블로킹 그래프 실행을 생성하는 방법입니다.


### 랭그래프와 함께


In [5]:
inputs = {"messages": [("human", "what's the weather in sf")]}
invoke_output = await graph.ainvoke(inputs)


In [6]:
for m in invoke_output["messages"]:
    m.pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_GOKlsBY2XKm7pZnmAzJweYDU)
 Call ID: call_GOKlsBY2XKm7pZnmAzJweYDU
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


### LangGraph 클라우드와 함께


In [7]:
# NOTE: We're not specifying the thread here -- this allows us to create a thread just for this run
wait_output = await client.runs.wait(None, "agent", input=inputs)


In [13]:
# we'll use this for pretty message formatting
from langchain_core.messages import convert_to_messages


In [9]:
for m in convert_to_messages(wait_output["messages"]):
    m.pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_pQJsT9uLG3nVppN8Dt2OhnFx)
 Call ID: call_pQJsT9uLG3nVppN8Dt2OhnFx
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


## 스트리밍


아래 예시는 부분 그래프 실행 결과를 스트리밍하는 `.stream() / .astream()` 메소드의 미러링 방법을 보여줍니다.  
참고: LangGraph의 `stream_mode=values/updates/debug`는 LangGraph Cloud에서 거의 동일하게 작동합니다(추가로 `metadata` / `end` 이벤트 유형과 함께 스트리밍된 청크가 있는 것을 제외하고).


### 랭그래프와 함께


In [10]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in graph.astream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_302y9671bqMkMcpLZOWLNAnq)
 Call ID: call_302y9671bqMkMcpLZOWLNAnq
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


### LangGraph 클라우드와 함께


In [11]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in client.runs.stream(
    None, "agent", input=inputs, stream_mode="values"
):
    if chunk.event == "values":
        messages = convert_to_messages(chunk.data["messages"])
        messages[-1].pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_NYVNSiBeF0oTAYnaDrlEAG7a)
 Call ID: call_NYVNSiBeF0oTAYnaDrlEAG7a
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


## 지속성


LangGraph에서는 그래프를 컴파일할 때 `checkpointer` 객체를 제공해야 그래프와의 상호작용(즉, 스레드) 간 상태를 지속적으로 유지할 수 있습니다. LangGraph Cloud에서는 checkpointer를 생성할 필요가 없습니다. 서버가 이미 이를 구현해 놓았습니다. 또한 클라이언트에서 스레드를 직접 관리할 수도 있습니다.


### 랭그래프와 함께


In [12]:
from langgraph.checkpoint.memory import MemorySaver


In [13]:
checkpointer = MemorySaver()
graph_with_memory = create_react_agent(model, tools, checkpointer=checkpointer)


In [14]:
inputs = {"messages": [("human", "what's the weather in nyc")]}
invoke_output = await graph_with_memory.ainvoke(
    inputs, config={"configurable": {"thread_id": "1"}}
)
invoke_output["messages"][-1].pretty_print()


================================== Ai Message ==================================

The weather in NYC might be cloudy.


In [15]:
inputs = {"messages": [("human", "what's it known for?")]}
invoke_output = await graph_with_memory.ainvoke(
    inputs, config={"configurable": {"thread_id": "1"}}
)
invoke_output["messages"][-1].pretty_print()


================================== Ai Message ==================================

New York City (NYC) is known for a variety of iconic landmarks, cultural institutions, and vibrant neighborhoods. Some of the most notable things NYC is known for include:

1. **Statue of Liberty**: A symbol of freedom and democracy.
2. **Times Square**: Famous for its bright lights, Broadway theaters, and bustling atmosphere.
3. **Central Park**: A large urban park offering a green oasis in the middle of the city.
4. **Empire State Building**: An iconic skyscraper with an observation deck offering panoramic views of the city.
5. **Broadway**: Renowned for its world-class theater productions.
6. **Wall Street**: The financial hub of the United States.
7. **Museums**: Including the Metropolitan Museum of Art, the Museum of Modern Art (MoMA), and the American Museum of Natural History.
8. **Diverse Cuisine**: A melting pot of culinary experiences from around the world.
9. **Cultural Diversity**: A rich tape

In [16]:
inputs = {"messages": [("human", "what's it known for?")]}
invoke_output = await graph_with_memory.ainvoke(
    inputs, config={"configurable": {"thread_id": "2"}}
)
invoke_output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Could you please specify what "it" refers to? Are you asking about a specific city, person, object, or something else?


In [17]:
# get the state of the thread
checkpointer.get({"configurable": {"thread_id": "2"}})


{'v': 1,
 'ts': '2024-06-22T02:31:49.722569+00:00',
 'id': '1ef303f9-4149-6b56-8001-a80d1e3c9dc6',
 'channel_values': {'messages': [HumanMessage(content="what's it known for?", id='ea0d1672-05e9-4d77-9dff-b33bd5c824e7'),
   AIMessage(content='Could you please specify what "it" refers to? Are you asking about a specific city, person, object, or something else?', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 57, 'total_tokens': 85}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0381dc0-d891-4203-8f77-3155ba17998c-0', usage_metadata={'input_tokens': 57, 'output_tokens': 28, 'total_tokens': 85})],
  'agent': 'agent'},
 'channel_versions': {'__start__': 2,
  'messages': 3,
  'start:agent': 3,
  'agent': 3},
 'versions_seen': {'__start__': {'__start__': 1},
  'agent': {'start:agent': 2},
  'tools': {}},
 'pending_sends': []}

### LangGraph Cloud 사용하기

이제 LangGraph Cloud를 사용하여 같은 작업을 재현해 보겠습니다. 체크포인터를 사용하는 대신, 백엔드에서 새 스레드를 생성하고 ID를 API에 전달하는 것을 주목하세요.


In [14]:
thread = await client.threads.create()


In [15]:
inputs = {"messages": [("human", "what's the weather in nyc")]}
wait_output = await client.runs.wait(thread["thread_id"], "agent", input=inputs)
convert_to_messages(wait_output["messages"])[-1].pretty_print()


================================== Ai Message ==================================

The weather in NYC might be cloudy.


In [16]:
inputs = {"messages": [("human", "what's it known for?")]}
wait_output = await client.runs.wait(thread["thread_id"], "agent", input=inputs)
convert_to_messages(wait_output["messages"])[-1].pretty_print()


================================== Ai Message ==================================

New York City (NYC) is known for a variety of iconic landmarks, cultural institutions, and vibrant neighborhoods. Some of the most notable things NYC is known for include:

1. **Statue of Liberty**: A symbol of freedom and democracy, located on Liberty Island.
2. **Times Square**: Known for its bright lights, Broadway theaters, and bustling atmosphere.
3. **Central Park**: A large urban park offering a green oasis in the middle of the city.
4. **Empire State Building**: An iconic skyscraper with an observation deck offering panoramic views of the city.
5. **Broadway**: Famous for its world-class theater productions and musicals.
6. **Wall Street**: The financial hub of the United States, home to the New York Stock Exchange.
7. **Museums**: Including the Metropolitan Museum of Art, the Museum of Modern Art (MoMA), and the American Museum of Natural History.
8. **Diverse Cuisine**: A melting pot of culinary

In [17]:
thread = await client.threads.create()


In [18]:
inputs = {"messages": [("human", "what's it known for?")]}
wait_output = await client.runs.wait(thread["thread_id"], "agent", input=inputs)
convert_to_messages(wait_output["messages"])[-1].pretty_print()


================================== Ai Message ==================================

Could you please specify what "it" refers to? Are you asking about a specific city, person, object, or something else?


In [19]:
# get the state of the thread
await client.threads.get_state(thread["thread_id"])


{'values': {'messages': [{'content': "what's it known for?",
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '381cd144-b360-4c7d-8177-e2634446993c',
    'example': False},
   {'content': 'Could you please specify what "it" refers to? Are you asking about a specific city, person, object, or something else?',
    'additional_kwargs': {'refusal': None},
    'response_metadata': {'token_usage': {'completion_tokens': 28,
      'prompt_tokens': 57,
      'total_tokens': 85,
      'completion_tokens_details': {'reasoning_tokens': 0}},
     'model_name': 'gpt-4o-2024-05-13',
     'system_fingerprint': 'fp_3537616b13',
     'finish_reason': 'stop',
     'logprobs': None},
    'type': 'ai',
    'name': None,
    'id': 'run-b23c6a05-d6c7-46fd-8b09-443a334feb6b-0',
    'example': False,
    'tool_calls': [],
    'invalid_tool_calls': [],
    'usage_metadata': {'input_tokens': 57,
     'output_tokens': 28,
     'total_tokens': 85}}]},
 'nex

## 중단점

### LangGraph와 함께


In [24]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in graph_with_memory.astream(
    inputs,
    stream_mode="values",
    interrupt_before=["tools"],
    config={"configurable": {"thread_id": "3"}},
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_cYp3BijeW2JNQ9RqJRdkrbMu)
 Call ID: call_cYp3BijeW2JNQ9RqJRdkrbMu
  Args:
    city: sf


In [25]:
async for chunk in graph_with_memory.astream(
    None,
    stream_mode="values",
    interrupt_before=["tools"],
    config={"configurable": {"thread_id": "3"}},
):
    chunk["messages"][-1].pretty_print()


================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is sunny!


### LangGraph 클라우드와 함께


지속성 예제와 유사하게, 상태를 유지하고 중단점에서 계속할 수 있도록 스레드를 생성해야 합니다.


In [26]:
thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=inputs,
    stream_mode="values",
    interrupt_before=["tools"],
):
    if chunk.event == "values":
        messages = convert_to_messages(chunk.data["messages"])
        messages[-1].pretty_print()


================================ Human Message =================================

what's the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_MVQEJtPYAj1nJ7J6YaCeLX8a)
 Call ID: call_MVQEJtPYAj1nJ7J6YaCeLX8a
  Args:
    city: sf


In [27]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=None,
    stream_mode="values",
    interrupt_before=["tools"],
):
    if chunk.event == "values":
        messages = convert_to_messages(chunk.data["messages"])
        messages[-1].pretty_print()


================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


## 스트리밍 이벤트


스트리밍 이벤트의 경우, LangGraph에서는 `CompiledGraph`에서 `.astream` 메서드를 사용해야 합니다. LangGraph Cloud에서는 `stream_mode="events"`를 통해 이 작업이 수행됩니다.


### LangGraph와 함께


In [22]:
from langchain_core.messages import AIMessageChunk

inputs = {"messages": [("human", "what's the weather in sf")]}
first = True
async for msg, metadata in graph.astream(inputs, stream_mode="messages"):
    if msg.content:
        print(msg.content, end="|", flush=True)

    if isinstance(msg, AIMessageChunk):
        if first:
            gathered = msg
            first = False
        else:
            gathered = gathered + msg

        if msg.tool_call_chunks:
            print(gathered.tool_calls)


[{'name': 'get_weather', 'args': {}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
[{'name': 'get_weather', 'args': {}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
[{'name': 'get_weather', 'args': {}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
[{'name': 'get_weather', 'args': {'city': ''}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_G6boP6Hj21glqPTqtFdTllUd', 'type': 'tool_call'}]
It's always sunny in sf|The| weather| in| San| Francisco| is| currently| sunny|.|

### LangGraph 클라우드와 함께


In [32]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in client.runs.stream(
    None, "agent", input=inputs, stream_mode="events"
):
    if chunk.event == "events" and chunk.data["event"] == "on_chat_model_stream":
        print(chunk.data["data"]["chunk"])


{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_JYWaAecaAV92cOlZwRHi9B7M', 'function': {'arguments': '', 'name': 'get_weather'}, 'type': 'function'}]}, 'response_metadata': {}, 'type': 'AIMessageChunk', 'name': None, 'id': 'run-855fec3d-15df-4ae8-b74d-208a0e463be9', 'example': False, 'tool_calls': [], 'invalid_tool_calls': [{'name': 'get_weather', 'args': '', 'id': 'call_JYWaAecaAV92cOlZwRHi9B7M', 'error': None}], 'usage_metadata': None, 'tool_call_chunks': [{'name': 'get_weather', 'args': '', 'id': 'call_JYWaAecaAV92cOlZwRHi9B7M', 'index': 0}]}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}, 'response_metadata': {}, 'type': 'AIMessageChunk', 'name': None, 'id': 'run-855fec3d-15df-4ae8-b74d-208a0e463be9', 'example': False, 'tool_calls': [{'name': '', 'args': {}, 'id': None}], 'invalid_tool_calls': [], 'usage_metadata': None, 'tool_call_chunks': [{'name': None, 'ar